In [83]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate
import torch

In [26]:
dataset = load_dataset("yelp_review_full")

Found cached dataset yelp_review_full (/Users/skelley/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)


  0%|          | 0/2 [00:00<?, ?it/s]

In [27]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(5120))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(512))

Loading cached processed dataset at /Users/skelley/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-1b229fb576f6410a.arrow
Loading cached processed dataset at /Users/skelley/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-769d3116751632a1.arrow
Loading cached shuffled indices for dataset at /Users/skelley/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-41a769da655b3614.arrow
Loading cached shuffled indices for dataset at /Users/skelley/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-db65ae019f1ce405.arrow


In [33]:
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", num_train_epochs=1, per_device_train_batch_size=16)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()
model.save_pretrained('./bert_fine_tuned')
torch.save(model.state_dict(),'./bert_fine_tuned/weights.pth')

loading configuration file config.json from cache at /Users/skelley/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.937006,0.585938


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 512
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)


Configuration saved in ./bert_fine_tuned/config.json
Model weights saved in ./bert_fine_tuned/pytorch_model.bin


In [81]:
loaded_model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased', num_labels=5)
#loaded_model.load_state_dict(torch.load('./bert_fine_tuned/weights.pth'))

loading configuration file config.json from cache at /Users/skelley/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "

In [74]:
def get_predictions(x):
    with torch.no_grad():
        p = model(torch.tensor(x['input_ids']), torch.tensor(x['token_type_ids']), torch.tensor(x['attention_mask']))
        return np.argmax(p.logits, axis=-1)

count = 0
correct = 0
for batch_number in range(int(512/8)):
    sample = small_eval_dataset[batch_number*16:(batch_number+1)*16]
    #prediction = loaded_model(torch.tensor(sample['input_ids']), torch.tensor(sample['token_type_ids']), torch.tensor(sample['attention_mask']))
    y_pred = get_predictions(sample)
    y = torch.tensor(sample['label'])
    with torch.no_grad():
        #y_pred = np.argmax(prediction.logits, axis=-1)
        print(y)
        print(y_pred)
        print(sum(y==y_pred))
        count = count + 8
        correct = correct + sum(y==y_pred)
        print(count, correct)
print(count, correct, correct/count)

tensor([2, 4, 1, 4, 3, 4, 2, 3, 2, 3, 0, 0, 3, 2, 2, 1])
tensor([4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4])
tensor(3)
8 tensor(3)
tensor([3, 1, 2, 2, 1, 2, 3, 1, 1, 3, 4, 0, 0, 2, 2, 2])
tensor([4, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])
tensor(1)
16 tensor(4)


KeyboardInterrupt: 

In [ ]:
def map_prediction(x):
    with torch.no_grad():
        #print(x['input_ids'], x['token_type_ids'], x['attention_mask'])
        p = loaded_model(torch.tensor(x['input_ids']), torch.tensor(x['token_type_ids']), torch.tensor(x['attention_mask']))
        prediction = np.argmax(p.logits, axis=-1).tolist()
        x['prediction'] = prediction
        print(prediction, x['prediction'], x['label'], sum(torch.tensor(prediction) == torch.tensor(x['label'])))
        return x

#train_result = small_train_dataset.map(lambda x: {"prediction": get_predictions(x, model)}, batch_size=8, batched=True)
#small_eval_dataset = small_eval_dataset.map(lambda x: {"prediction": get_predictions(x)}, batch_size=8, batched=True)
#small_eval_dataset = small_eval_dataset.map(lambda x: {"prediction": x['label']}, batch_size=8, batched=True
eval_result = small_eval_dataset.map(map_prediction, batch_size=2, batched=True)
#print(sum(torch.tensor(train_result[:]['prediction'])==torch.tensor(train_result[:]['label'])))
print(sum(torch.tensor(eval_result[:]['prediction'])==torch.tensor(eval_result[:]['label'])))

In [47]:
eval_result['prediction'][0:20], eval_result['label'][0:20], sum(torch.tensor(eval_result['prediction'][0:20]) - torch.tensor(eval_result['label'][0:20]))

([3, 0, 0, 4, 0, 4, 0, 0, 4, 4, 4, 0, 4, 4, 4, 4, 4, 2, 4, 0],
 [2, 4, 1, 4, 3, 4, 2, 3, 2, 3, 0, 0, 3, 2, 2, 1, 3, 1, 2, 2],
 tensor(5))

In [40]:
s = small_train_dataset[0:10]
print(s['label'])
print(get_predictions(s))

[4, 2, 4, 0, 2, 2, 3, 0, 0, 4]
tensor([4, 4, 4, 0, 2, 1, 4, 0, 0, 4])


In [54]:
trainer_output = trainer.predict(small_eval_dataset)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 512
  Batch size = 8


In [55]:
trainer_output

PredictionOutput(predictions=array([[-2.0485823 , -1.4629366 ,  0.03176205,  1.9571215 ,  2.1546638 ],
       [-2.194212  , -0.20936851,  1.4527514 ,  1.0241582 , -0.5653589 ],
       [ 1.9212087 ,  0.20988111, -1.1255125 , -0.83040965, -0.38002148],
       ...,
       [-1.5981525 ,  0.6202789 ,  1.5973585 ,  0.08563099, -1.6112113 ],
       [ 0.00967137,  1.21231   ,  0.75466865, -1.0023788 , -2.2013545 ],
       [-0.4652198 ,  0.01005547,  0.14590707, -0.07891287, -0.33450198]],
      dtype=float32), label_ids=array([2, 4, 1, 4, 3, 4, 2, 3, 2, 3, 0, 0, 3, 2, 2, 1, 3, 1, 2, 2, 1, 2,
       3, 1, 1, 3, 4, 0, 0, 2, 2, 2, 1, 3, 4, 0, 0, 1, 3, 2, 0, 2, 0, 0,
       3, 0, 3, 2, 3, 0, 1, 1, 3, 3, 4, 4, 1, 4, 1, 3, 1, 0, 0, 1, 4, 1,
       4, 3, 2, 4, 1, 0, 3, 3, 4, 1, 2, 1, 0, 4, 4, 4, 2, 3, 3, 1, 4, 0,
       4, 2, 3, 0, 0, 0, 3, 4, 0, 0, 1, 4, 4, 0, 0, 1, 1, 0, 4, 2, 2, 1,
       1, 4, 0, 4, 0, 3, 2, 0, 4, 4, 4, 2, 0, 0, 0, 1, 3, 0, 2, 0, 3, 2,
       2, 2, 0, 3, 4, 3, 0, 1, 0, 1, 0, 0, 4

In [60]:
trainer_labels = np.argmax(trainer_output.predictions, axis=-1)
sum(torch.tensor(trainer_output.label_ids) == torch.tensor(trainer_labels))

tensor(300)

In [49]:
trainer.save_model('./bert_fine_tuned/trainer/')

Saving model checkpoint to ./bert_fine_tuned/trainer/
Configuration saved in ./bert_fine_tuned/trainer/config.json
Model weights saved in ./bert_fine_tuned/trainer/pytorch_model.bin


In [93]:
loaded_model = BertForSequenceClassification.from_pretrained('./bert_fine_tuned/trainer/')
loaded_model.load_state_dict(torch.load('./bert_fine_tuned/weights.pth'))

loading configuration file ./bert_fine_tuned/trainer/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
 

<All keys matched successfully>

In [95]:
def get_predictions(x):
    with torch.no_grad():
        p = model(torch.tensor(x['input_ids']), torch.tensor(x['token_type_ids']), torch.tensor(x['attention_mask']))
        return np.argmax(p.logits, axis=-1)
    
sample = small_eval_dataset[0:25]
get_predictions(sample)

tensor([4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4,
        4])

In [97]:
sample['label']

[2, 4, 1, 4, 3, 4, 2, 3, 2, 3, 0, 0, 3, 2, 2, 1, 3, 1, 2, 2, 1, 2, 3, 1, 1]